# 뉴스 기사 이슈탐지 시스템
1. 데이터 확인
2. 데이터를 article 단위로 분리 
3. 요구사항 정의 
4. 시스템 설계 
5. 시스템 구현(선택사항)

# 하나의 문서(기사) 처리 
- 엑셀파일로 정리된 데이터를 불러 news에 해당되는 처리 

In [112]:
import numpy as np # linear algebra
import pandas as pd # data processing
import os
import re
import sklearn
import sklearn.model_selection as ms
import sklearn.feature_extraction.text as text
import sklearn.naive_bayes as nb
import matplotlib.pyplot as plt
%matplotlib inline

import konlpy
import nltk
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

from konlpy.tag import Twitter, Kkma, Mecab, Hannanum
from soynlp.tokenizer import RegexTokenizer
t = Twitter()
kkma = Kkma()
hannanum = Hannanum()

# set directory with yours
base_dir = 'c:/Users/user/git/saltlux/[P]textPreprocessing'
excel_file = 'article_content_train.xlsx'
excel_dir = os.path.join(base_dir, excel_file)

# read a excel file and make it as a DataFrame

df_from_excel = pd.read_excel(excel_dir, sheet_name = 'Zinynews Content', 
                              dtype = {'아티클 아이디': str, '제목': str, '내용': str}, # dictionary type
                              na_values = 'NaN')

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [97]:
# 기존의 데이터
df_from_excel.shape

(2838, 3)

In [98]:
df_news = df_from_excel[df_from_excel['아티클 아이디'].str.contains("news")]
df_news.head(5)

,아티클 아이디,제목,내용
862,news-1005362483379086974-0-kor,"아이유, 깜짝 싱글 '삐삐' 발표…10주년 기념",▲ 가수 아이유가 이달 '깜짝 신곡'을 발표한다.ⓒ카카오M\n가수 아이유가 이달 '...
863,news-1012971976832336506-0-kor,"페라리, 세상에 단 한대 뿐인 'SP3JC'…LA오토쇼서 선봬",SP3JC (사진=페라리 공식 홈페이지)\n[이데일리 김민정 기자] 페라리가 29일...
864,news-1025890183670493361-0-kor,"'연애의맛' 이필모, 서수연과 데이트..연애고수 거듭났다- 스타뉴스",/사진=TV조선 '연애의 맛' 방송화면 캡처\n배우 이필모가 서수연과 데이트를 하며...
865,news-1037931898441396700-0-kor,한국미래기술 양진호회장에 폭행당한 위디스크 전 직원 섬에서 은둔 생활 “보복 당할 ...,사진=뉴스타파\n한국미래기술 양진호 회장으로부터 폭행을 당한 위디스크 전 직원이 섬...
866,news-1065838178259793765-0-kor,카카오 2018 멜론뮤직어워드 DAY…'출연 라인업?',▲ ‘카카오와 함께하는 2018 멜론뮤직어워드(이하 2018 멜론뮤직어워드)’이 드...


# 전처리 (명사, 명사구만 추출)

정규화 (normalization)
토큰화 (tokenization)
어근화 (stemming)
어구추출 (phrase extraction)

In [5]:
# 유니크한 값
df_news['제목'].nunique(), df_news['내용'].nunique()

(1971, 1950)

In [100]:
print('총 샘플의 수 :',len(df_news))

총 샘플의 수 : 1976


In [101]:
df_news.columns.values

array(['아티클 아이디', '제목', '내용'], dtype=object)

In [102]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1976 entries, 862 to 2837
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   아티클 아이디  1976 non-null   object
 1   제목       1976 non-null   object
 2   내용       1976 non-null   object
dtypes: object(3)
memory usage: 61.8+ KB


In [108]:
title = str(df_news['제목'])
content = str(df_news['내용'])

'862                            아이유, 깜짝 싱글 \'삐삐\' 발표…10주년 기념\n863                    페라리, 세상에 단 한대 뿐인 \'SP3JC\'…LA오토쇼서 선봬\n864                 \'연애의맛\' 이필모, 서수연과 데이트..연애고수 거듭났다- 스타뉴스\n865     한국미래기술 양진호회장에 폭행당한 위디스크 전 직원 섬에서 은둔 생활 “보복 당할 ...\n866                        카카오 2018 멜론뮤직어워드 DAY…\'출연 라인업?\'\n                              ...                        \n2833    [No.1 문화신문 [뉴스컬처]] [인방]아프리카티비 BJ지여닝, 바비인형 미모 ‘눈길’\n2834                                한동민에게 전부 쏠린 PS 우주의 기운\n2835             "홍수 伊항만 ‘화재’…주차된 슈퍼카 마세라티 수백대 불타"- 헤럴드경제\n2836                                  삼성전자, 자사주 22兆 소각에도…\n2837                   \'동네사람들\' 마동석표 액션이 스릴러를 만났을 때 - 노컷뉴스\nName: 제목, Length: 1976, dtype: object'

In [105]:
title_han = hannanum.nouns(title)
print(len(title_han))
title_han[:10]

71


['862', '아이유', '싱글', '삐삐', '발표…10주년', '기념', '863', '페라리,', '세상', '한대']

In [106]:
content_han = hannanum.nouns(content)
print(len(content_han))
content_han

92


['862', '▲', '가수', '아이유', '이달', '신곡', '발표', 'ⓒ카카오M\\n가수', '아이유', '이달']

# 토큰화

In [109]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [110]:
X_train = []
i=0;
for sentence in df_news['제목']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [113]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [114]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 7344
등장 빈도가 2번 이하인 희귀 단어의 수: 5349
단어 집합에서 희귀 단어의 비율: 72.83496732026144
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 20.716160479940008


In [115]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

NameError: name 'vocab_size' is not defined

In [63]:
sample_title = df_news['제목']
sample_title

862                            아이유, 깜짝 싱글 '삐삐' 발표…10주년 기념
863                    페라리, 세상에 단 한대 뿐인 'SP3JC'…LA오토쇼서 선봬
864                 '연애의맛' 이필모, 서수연과 데이트..연애고수 거듭났다- 스타뉴스
865     한국미래기술 양진호회장에 폭행당한 위디스크 전 직원 섬에서 은둔 생활 “보복 당할 ...
866                        카카오 2018 멜론뮤직어워드 DAY…'출연 라인업?'
                              ...                        
2833    [No.1 문화신문 [뉴스컬처]] [인방]아프리카티비 BJ지여닝, 바비인형 미모 ‘눈길’
2834                                한동민에게 전부 쏠린 PS 우주의 기운
2835             "홍수 伊항만 ‘화재’…주차된 슈퍼카 마세라티 수백대 불타"- 헤럴드경제
2836                                  삼성전자, 자사주 22兆 소각에도…
2837                   '동네사람들' 마동석표 액션이 스릴러를 만났을 때 - 노컷뉴스
Name: 제목, Length: 1976, dtype: object

In [16]:
sample_content = df_news['내용']
sample_content

'출처=이미지투데이\n[이데일리 뉴스속보팀] 일본에서 범죄조직원인 야쿠자들이 어린이들을 불러 핼러윈 행사를 개최해 논란이 되고 있다.\n야쿠자들은 “아이들이 원해서”라고 행사 개최 이유를 설명했지만, 지역 주민들은 집회를 열어 “과자를 나눠주는 일을 중단하라”고 외쳤다.\n31일 NHK에 따르면 일본 최대의 야쿠자 조직 야마구치구미(山口組)는 31일 효고(兵庫)현 고베(神戶)시에 위치한 이 조직의 본부에서 핼러윈 데이 행사를 열고 아이들에게 과자를 나눠줬다.\n야마구치구미는 호박이나 귀신 장식 등으로 핼러윈 분위기를 낸 채 행사를 벌였고, 참가자 중에는 어린 딸을 데리고 온 여성의 모습도 보였다고 NHK는 설명했다.\n야마구치구미는 핼러윈 데이 행사 개최 이유에 대해 “아이들이 좋아하기 때문”이라고 설명하고 있지만, 경찰은 야쿠자를 배척하는 분위기가 확산하자 지역 주민들을 회유하려는 목적이 있는 것으로 보고 있다.\n하지만 올해 야마구치구미의 핼로윈 행사는 지역 주민들이 본부 앞에 모여 항의 집회를 열고 거세게 반발하는 가운데 개최됐다는 점에서 예년과는 달랐다.\n주민 100여명은 ‘폭력단 추방’이라고 쓰인 깃발을 든 채 “야쿠자 조직의 핼러윈 행사 개최가 아이들의 건전한 성장에 악영향을 주고 있다”고 비판했다.\n항의집회에 참석한 한 남성은 “아이들은 잘 모른 채 야쿠자로부터 과자를 받고 있다. 어른들이 이를 저지해야 한다”고 말했다.\n변호사인 다른 남성은 “핼러윈 행사의 개최에 사용된 돈은 범죄 수익에서 온 것일 수 있다”며 “아이들에게 (야쿠자가 주는) 과자를 받게 하는 것은 야쿠자가 위법한 자금을 획득하기 위해 하고 있는 활동을 긍정하는 것이 된다”고 경계했다.\n현재 일본 정부의 법률이나 효고현의 조례에는 이런 식으로 야쿠자들이 아이들을 대상으로 행사를 개최하는 것을 금지하는 규정은 없다.\n'

In [60]:
tokened_title = tokenizer.tokenize(sample_title)
tokened_title

['日', '야쿠자들', ',', '어린이', '초청', "'", '핼러윈', '행사', "'…", '주민', '항의']

In [61]:
tokened_content = tokenizer.tokenize(sample_content)
tokened_content[:10]

['출처', '=', '이미지투데이', '[', '이데일리', '뉴스속보팀', ']', '일본에서', '범죄조직원인', '야쿠자들이']

In [62]:
print(len(tokened_title))
print(len(tokened_content))

11
261


In [20]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    return text

In [71]:
%time sentences = df_news['내용'].apply(preprocessing)

Wall time: 4.06 ms


In [72]:
%time tokens = content.apply(tokenizer.tokenize)
tokens[:10]

Wall time: 2.76 s


862    [▲, 가수, 아이유가, 이달, ', 깜짝, 신곡, ', 을, 발표한다, .ⓒ, 카...
863    [SP, 3, JC, (, 사진, =, 페라리, 공식, 홈페이지, ), [, 이데일...
864    [/, 사진, =, TV, 조선, ', 연애의, 맛, ', 방송화면, 캡처, 배우,...
865    [사진, =, 뉴스타파, 한국미래기술, 양진호, 회장으로부터, 폭행을, 당한, 위디...
866    [▲, ‘, 카카오와, 함께하는, 2018, 멜론뮤직어워드, (, 이하, 2018,...
867    [2018, /, 09, /, 30, 16, :, 30, 높은, 음식값에, 손님, ...
868    [", 종합금융투자사업자, 제도, 발단, ", ", 소형사, 과감한, 특화전략, 필...
869    [눌림목, 매매기업의, 달인, !, 매일, 안전하게, 수익을, 내고, 싶은, 투자자...
870    [", 한국에, 투자하는, 기업, 적어질, 것, ", ", 분노를, 넘어서, 기, ...
871    [박범계, 더불어민주당, 의원, (, 사진, =, 연합뉴스, ), [, 이데일리, ...
Name: 내용, dtype: object

In [73]:
tokens[862]

['▲',
 '가수',
 '아이유가',
 '이달',
 "'",
 '깜짝',
 '신곡',
 "'",
 '을',
 '발표한다',
 '.ⓒ',
 '카카오',
 'M',
 '가수',
 '아이유가',
 '이달',
 "'",
 '깜짝',
 '신곡',
 "'",
 '을',
 '발표한다',
 '.',
 '아이유',
 '소속사',
 '카카오',
 'M',
 '은',
 '"',
 '아이유가',
 '오는',
 '10',
 '일',
 '데뷔',
 '10',
 '주년을',
 '기념해',
 '깜짝',
 '디지털',
 '싱글',
 "'",
 '삐삐',
 "'",
 '를',
 '발표한다',
 '"',
 '고',
 '1',
 '일',
 '밝혔다',
 '.',
 '지난해',
 '9',
 '월',
 ',',
 '두',
 '번째',
 '리메이크',
 '음반',
 "'",
 '꽃갈피',
 '둘',
 "'",
 '이후',
 '1',
 '년',
 '여',
 '만에',
 '신곡',
 '발표에',
 '나서는',
 '아이유는',
 '올가을',
 ',',
 '데뷔',
 '10',
 '주년을',
 '기념하는',
 '스페셜',
 '싱글을',
 '통해',
 '오랜만에',
 '음악',
 '팬들을',
 '찾는다',
 '.',
 '소속사는',
 '"',
 '올해',
 '공식적인',
 '신곡발매',
 '계획은',
 '잡혀있지',
 '않았다',
 '"',
 '며',
 '"',
 '아이유가',
 '데뷔',
 '10',
 '주년을',
 '맞이한',
 '특별한',
 '한',
 '해인',
 '만큼',
 '많은',
 '팬분들이',
 '아이유의',
 '신곡을',
 '기다려',
 '주셔서',
 '깜짝',
 '발매를',
 '결정하게',
 '됐다',
 '"',
 '고',
 '전했다',
 '.',
 '이어',
 '"',
 '신곡',
 "'",
 '삐삐',
 "'",
 '는',
 '그간',
 '선보이지',
 '않았던',
 '신선한',
 '콘셉트의',
 '곡으로',
 '데뷔',
 '10',
 '주년에',
 '맞추어',
 '10',
 '월',


In [80]:
stopwords_kr = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '▲', ' ', '.', '.']


In [90]:

from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

def displayWordCloud(data = None, backgroundcolor = 'white', width=800, height=600 ):
    wordcloud = WordCloud(font_path = '/Library/Fonts/NanumBarunGothic.ttf', 
                        stopwords = stopwords_kr, background_color = backgroundcolor, 
                        width = width, height = height).generate(data)
    plt.figure(figsize = (15 , 10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [91]:
from soynlp.noun import LRNounExtractor

In [94]:
%%time
noun_extractor = LRNounExtractor(verbose=True)
noun_extractor.train(sentences)
nouns = noun_extractor.extract()
nouns

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (29719, 15060) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 6629 nouns are extracted
Wall time: 5.77 s


{'우스': NounScore_v1(frequency=2, score=0.951425, known_r_ratio=0.5),
 '반납': NounScore_v1(frequency=10, score=0.821517, known_r_ratio=0.42857142857142855),
 '제정': NounScore_v1(frequency=16, score=0.99880725, known_r_ratio=0.4),
 '청탁': NounScore_v1(frequency=4, score=0.75126, known_r_ratio=1.0),
 '급감': NounScore_v1(frequency=9, score=0.9945606666666666, known_r_ratio=0.8571428571428571),
 '미수': NounScore_v1(frequency=14, score=0.99972, known_r_ratio=0.3333333333333333),
 '불구': NounScore_v1(frequency=66, score=0.9925110204081633, known_r_ratio=0.765625),
 '휴일': NounScore_v1(frequency=9, score=0.7521365, known_r_ratio=0.5714285714285714),
 '노출': NounScore_v1(frequency=38, score=0.9723511111111112, known_r_ratio=0.6),
 '입사': NounScore_v1(frequency=15, score=0.9977777, known_r_ratio=0.8333333333333334),
 '기재': NounScore_v1(frequency=36, score=0.9965005999999998, known_r_ratio=0.15151515151515152),
 '검사': NounScore_v1(frequency=79, score=0.823633731707317, known_r_ratio=0.7884615384615384),
 

In [93]:
import matplotlib.pyplot as plt

from matplotlib import font_manager

font_fname = 'C:/Windows/Fonts/NanumGothic.ttf'

font_family = font_manager.FontProperties(fname=font_fname).get_name()

plt.rcParams["font.family"] = font_family

# 추출된 명사를 찍어봅니다.
%time displayWordCloud(' '.join(nouns))

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Windows/Fonts/NanumGothic.ttf'

In [27]:
# word2vec 모델 학습에 로그를 찍을 수 있도록 합니다.
import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', 
    level=logging.INFO)

In [28]:
# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
model = word2vec.Word2Vec(tokens, min_count=1)

model

2020-09-06 18:14:46,704 : INFO : collecting all words and their counts
2020-09-06 18:14:46,705 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-09-06 18:14:46,834 : INFO : collected 105263 word types from a corpus of 579956 raw words and 1976 sentences
2020-09-06 18:14:46,835 : INFO : Loading a fresh vocabulary
2020-09-06 18:14:47,114 : INFO : effective_min_count=1 retains 105263 unique words (100% of original 105263, drops 0)
2020-09-06 18:14:47,115 : INFO : effective_min_count=1 leaves 579956 word corpus (100% of original 579956, drops 0)
2020-09-06 18:14:47,422 : INFO : deleting the raw counts dictionary of 105263 items
2020-09-06 18:14:47,425 : INFO : sample=0.001 downsamples 23 most-common words
2020-09-06 18:14:47,427 : INFO : downsampling leaves estimated 525868 word corpus (90.7% of prior 579956)
2020-09-06 18:14:47,665 : INFO : estimated required memory for 105263 words and 100 dimensions: 136841900 bytes
2020-09-06 18:14:47,665 : INFO : resettin

In [47]:
# 모델 이름을 지정하고 저장한다.
model_name = '1minwords'
model.save(model_name)

2020-09-06 18:29:25,074 : INFO : saving Word2Vec object under 1minwords, separately None
2020-09-06 18:29:25,079 : INFO : storing np array 'vectors' to 1minwords.wv.vectors.npy
2020-09-06 18:29:25,130 : INFO : not storing attribute vectors_norm
2020-09-06 18:29:25,131 : INFO : storing np array 'syn1neg' to 1minwords.trainables.syn1neg.npy
2020-09-06 18:29:25,187 : INFO : not storing attribute cum_table
2020-09-06 18:29:25,399 : INFO : saved 1minwords


In [ ]:
# 단어 사전 수
len(model.wv.vocab)

In [48]:
# 단어 사전에서 상위 10개만 보기
vocab = model.wv.vocab
sorted(vocab, key=vocab.get, reverse=True)[:30]

['.',
 ',',
 '(',
 '"',
 ')',
 "'",
 '있다',
 '일',
 '“',
 '1',
 '‘',
 '”',
 '’',
 '수',
 ':',
 '=',
 '고',
 '2',
 '년',
 '·',
 '등',
 '3',
 '월',
 '기자',
 '%',
 '5',
 ']',
 '있는',
 '/',
 '이']

In [ ]:
# Counter로 자주 등장하는 단어 보기
from collections import Counter
dict(Counter(vocab).most_common(20))

In [ ]:
# 가장 적게 등장하는 단어
min(vocab, key=vocab.get)

In [ ]:
model.wv['주식']

In [ ]:
model.wv.most_similar('주식')

In [ ]:
model.wv.most_similar('부동산')

In [ ]:
# 유사도가 없는 단어 추출
model.wv.doesnt_match('부동산 증권 현금 코스닥 코스피'.split())

In [ ]:
# 가장 유사한 단어를 추출
model.wv.most_similar('주식')

In [ ]:
# 가장 유사한 단어를 추출
model.wv.most_similar('공매도')

In [ ]:
model.wv.most_similar(positive=['주식', '증권'], negative=['현금'])

In [ ]:
model.wv.similarity('주식', '부동산')

In [ ]:
model.wv.similarity('주식', '투자')

In [ ]:
model.wv.similarity('주식', '증권')

In [ ]:
model.wv.similarity('주식', '현금')

In [ ]:
model.wv.similarity('삼성', '증권')

In [ ]:
model.wv.similarity('삼성증권', '공매도')

In [ ]:
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
import gensim 
import gensim.models as g

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

model_name = '1minwords'
model = g.Doc2Vec.load(model_name)

In [ ]:
# Doc2Vec으로 만든 모델을 가져옴
vocab = list(model.wv.vocab)
# 모델의 단어를 피처로 지정해 준다.
X = model[vocab]

print('Feature의 길이 {}'.format(len(X)))
print(X[0][:10])
tsne = TSNE(n_components=2)

# 모든 단어를 그래프에 출력하면 글자가 너무 많이 겹치기 때문에 일부 단어에 대해서만 시각화
X_tsne = tsne.fit_transform(X[:150,:])

In [ ]:
df = pd.DataFrame(X_tsne, index=vocab[:150], columns=['x', 'y'])
df.shape

In [ ]:
df.head(20)

In [ ]:
fig = plt.figure()
fig.set_size_inches(40, 20)
ax = fig.add_subplot(1, 1, 1)

ax.scatter(df['x'], df['y'])

for word, pos in df.iterrows():
    ax.annotate(word, pos, fontsize=30)
plt.show()